> **Petit exercice d'info sympa : La fourmi de Langton**





Quelques outils que je vous fournis (voir explications ci-dessous) :

In [ ]:
!pip install matplotlib

import matplotlib.pyplot as plt
import matplotlib.animation as am

class tableau():
	
	def __init__(self, tab, d, x_b = 0, y_b = 0):
		self.d_val = d
		self.x_b = x_b
		self.y_b = y_b
		self.w = len(tab[0])
		self.h = len(tab)
		self.id = tab
	
	def show(self, inter = 'nearest'):
		plt.figure()
		plt.imshow(self.id, cmap = 'gray', interpolation = inter)
		plt.pause(0.1)
	
	def copy(self):
		temp = [[self.id[i][j] for j in range(self.w)] for i in range(self.h)]
		return tableau(temp, self.d_val, self.x_b, self.y_b)
	
	def reverse_y(self):
		temp = []
		for i in range(self.h):
			temp.append(self.id[self.h - i - 1])
		self.id = temp
	
	def add_c(self, c):
		if c>0:
			for j in range(self.h):
				self.id[j] = self.id[j] + [self.d_val]
		else:
			for j in range(self.h):
				self.id[j] = [self.d_val] + self.id[j]
			self.x_b -= 1
		self.w += 1
	
	def add_l(self, c):
		if c>0:
			self.id = self.id + [[self.d_val for i in range(self.w)]]
		else:
			self.id = [[self.d_val for i in range(self.w)]] + self.id
			self.y_b -= 1
		self.h += 1
	
	def elt(self,x,y):
		if x<self.x_b or y<self.y_b or x>=self.x_b+self.w or y>=self.y_b+self.h:
			return self.d_val
		else:
			return self.id[y-self.y_b][x-self.x_b]
	
	def chg_elt(self,x,y,e):
		if x<self.x_b or x>=self.x_b+self.w:
			self.add_c(x-self.x_b)
			self.chg_elt(x,y,e)
		elif y<self.y_b or y>=self.y_b+self.h:
			self.add_l(y-self.y_b)
			self.chg_elt(x,y,e)
		else:
			self.id[y-self.y_b][x-self.x_b] = e
	
	def include(self,left,right,bottom,top):
		self.chg_elt(left,bottom,self.elt(left,bottom))
		self.chg_elt(right,top,self.elt(right,top))

def affiche_joli(t):
  for e in t:
    print(e)

En résumé rapide : La classe "tableau" mime un tableau infini.

Exemple :

je prends l'instruction suivante :

In [ ]:
t = tableau([[1]],1)

Par défaut, toutes les valeurs de t valent la valeur du deuxième argument passé à la fonction tableau, ici 1.

On accède à l'élément d'abscisse x et d'ordonnée y par la méthode t.elt(x,y) (équivalent donc à t[y][x] pour un tableau python "classique" (c'est-à-dire une liste de listes, qu'on appellera par la suite "array" pour éviter les confusions avec la classe tableau introduite)).

Exemple :

In [ ]:
print(t.elt(-14,17))

Evidemment, étant donné que même la mémoire des serveurs que nous sommes en train d'utiliser n'est pas infinie, les valeurs de t ne sont donc pas stockées une à une dans la mémoire.

L'astuce consiste à dire que les valeurs de t sont toutes égales à 1 (pas besoin de les stocker une à une donc), sauf pour un nombre fini d'entre elles, qui seront stockées dans un array, auquel on peut accéder par la méthode t.id.

Initialement, t.id vaut le premier argument passé à la fonction tableau, ici [[1]].

In [ ]:
print(t.id)

Ensuite, lorsque l'on veut modifier des cases (avec la méthode t.chg_elt(x,y,e), qui assigne à l'élément d'abscisse x et d'ordonnée y la valeur e, équivalent donc à t[y][x] = e pour un array) qui ne sont pas stockées dans t.id, t.id va simplement être agrandi pour contenir ces cases :

In [ ]:
t.chg_elt(2,3,78)
print(t.elt(2,3))
affiche_joli(t.id)

Repérons sur t.id où est le 0:

In [ ]:
t.chg_elt(0,0,0)
affiche_joli(t.id)

Encore un exemple de modification du tableau :

In [ ]:
t.chg_elt(-2,-3,65)
print(t.elt(-2,-3))
affiche_joli(t.id)

On peut agrandir "manuellement" t.id avec t.include(left,right,bottom,top) qui va faire en sorte que t.id stocke les valeurs incluses dans un rectangle délmité par left à gauche, right à droite, bottom en bas et top en haut :

In [ ]:
t.include(-3,2,-4,3)
affiche_joli(t.id)

Afin d'éviter des problèmes, on a doté la classe tableau d'une méthode .copy :

In [ ]:
a = tableau([[0]],0)
b = a #maintenant b = a, mais ce sera tout le temps vrai ! Les changements appliqués à a se répercuteront sur b. Ce problème existe aussi avec les arrays.
c = a.copy() #Evite ce problème

a.chg_elt(0,0,75)
affiche_joli(a.id)
print("\n")
affiche_joli(b.id)
print("\n")
affiche_joli(c.id)

Dans le cas d'un tableau constitué uniquement de 0 et de 1, la méthode t.show() donne une représentation de ce t.id (0 = noir, 1 = blanc) :

In [ ]:
new = tableau([[1]],1)
new.chg_elt(0,1,0)
new.chg_elt(1,0,0)
affiche_joli(new.id)
new.show()

Maintenant entrons dans le vif du sujet. La fourmi de Langton répond à des règles simples :

Elle évolue sur un tableau à cases carrées, initialement toutes blanches.

A chaque étape :

1) Elle change la couleur de la case sur laquelle elle est : Une case blanche devient noire et inversement.

2) Elle avance

3) Si la case sur laquelle elle arrive est blanche, elle tourne à droite, si la case est noire, elle tourne à gauche.

De quelles informations va-t-on avoir besoin pour établir la trajectoire de la fourmi ?

Autrement dit, de quelles informations va-t-on avoir besoin pour passer de l'étape n à l'étape n+1 ?

Comment peut-on afficher la fourmi ? Faire des tests.

Comment va-t-on coder sa direction, en pensant au fait que l'on doive pouvoir changer sa position facilement en fonction de sa direction ?

Finalement, que va-t-on utiliser pour représenter une étape de la fourmi ?

Faire se déplacer la fourmi, en utilisant sa direction, sa position, et le tableau sur lequel elle se déplace.

Comment la faire changer de direction ? Comment coder le fait de tourner à droite/gauche ?

Faire des tests en faisant se déplacer la fourmi.

Comment changer la couleur de la case sur laquelle la fourmi passe ?

Coder la fonction faisant passer la fourmi de l'étape n à l'étape n+1.

Coder une fonction faisant aboutir la fourmi à l'étape n, puis une autre affichant le tableau à l'étape n. La tester pour diverses valeurs de n (100, 1000, 10000)

Un nouvel outil :

In [ ]:
def video(l, ms):
	fig, ax = plt.subplots()
	ax.imshow(l[0],cmap='gray')
	ims = [[ax.imshow(l[i],cmap='gray',animated=True)] for i in range(len(l))]
	ani = am.ArtistAnimation(fig, ims, interval=ms, blit=True, repeat_delay=5000)
	plt.show()

La fonction video prend en argument une liste d'arrays de mêmes tailles, un temps par image (en millisecondes) et affiche le film constitué des images correspondantes (avec 1 = blanc, 0 = noir).

Nous voilà face à un problème :

Nous pouvons avoir accès aux tableaux de chaque étape de la fourmi, et en les enregistrant dans une liste, on pourrait montrer le film correpondant et ainsi voir notre fourmi avancer. Mais ces tableaux ne sont pas tous de la même taille !

Comment peut-on alors faire pour obtenir des tableaux de même taille ?

Coder la fonction donnant la liste des bons tableaux (de mêmes tailles) jusqu'à l'étape n.

Coder la fonction donnant le film jusqu'à l'étape n, en gardant le nombre de millisecondes par image en argument.

Aller ensuite sur https://repl.it, "start coding", puis choisissez Python, puis "create repl".

Tester ensuite (sur repl.it, bien penser à inclure toutes les étapes précédentes nécessaires, et enlever "!pip install matplotlib" dans la première cas de code) pour n = 200, et ms = 400, avec ms le nombre de millisecondes par image.

Coder la fonction donnant les bons tableaux de l'étape n_d jusqu'à l'étape n_f.

Coder la fonction donnant le film des étapes n_d à n_f, en gardant le nombre de millisecondes par image en argument.

Tester ensuite pour n_d = 10100, n_f = 10500, ms = 500.